#### Генерация и определение эллиптической кривой


In [2]:
#from __future__ import annotations # PEP 563: Postponed Evaluation of Annotations
from dataclasses import dataclass # https://docs.python.org/3/library/dataclasses.html I like these a lot

@dataclass
class Curve:
    """
    Для для эллиптической кривой над конечным полем целых чисел.
    """
    p: int # модуль кривой
    a: int
    b: int

# В secp256k1 a = 0, b = 7, то есть уравнение кривой y^2 = x^3 + 7 (mod p)
bitcoin_curve = Curve(
    p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F, # 2^256-2^32-2^9-2^8-2^7-2^6-2^4-1
    a = 0x0000000000000000000000000000000000000000000000000000000000000000, # a = 0
    b = 0x0000000000000000000000000000000000000000000000000000000000000007, # b = 7
)

Класс точка на кривой

In [3]:
@dataclass
class Point:
    """ Точка (x,y) на плоскости на эллиптической кривой """
    curve: Curve
    x: int
    y: int

G = Point(
    bitcoin_curve,
    x = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798,
    y = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8,
)

# Проверим что наша точка G лежит на кривой secp256k1
print("Точка G лежит на кривой?: ", (G.y**2 - G.x**3 - 7) % bitcoin_curve.p == 0)

# Другие случайно сгенерированные кривые скорее всего не будут лежать на кривой
import random
random.seed(1337)
x = random.randrange(0, bitcoin_curve.p)
y = random.randrange(0, bitcoin_curve.p)
print("Случайная точка лежит на кривой?: ", (y**2 - x**3 - 7) % bitcoin_curve.p == 0)

Точка G лежит на кривой?:  True
Случайная точка лежит на кривой?:  False


Класс для важных полей на эллиптической кривой

In [4]:
@dataclass
class Generator:
    """
    A generator over a curve: an initial point and the (pre-computed) order
    """
    G: Point     # начальная точка на кривой
    n: int       # 0*G = n*G = INF

bitcoin_gen = Generator(
    G = G,
    # порядок кривой secp256k1
    n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141,
)

Секретный ключ - число в диапазоне от 1 до порядка кривой

In [5]:
secret_key = random.randrange(1, bitcoin_gen.n) # генерируем случайный ключ, то есть просто число от 1 до n
assert 1 <= secret_key < bitcoin_gen.n
print(secret_key)

88179880527772193127991559081690516360423762505570950046565861475646252139708


Определение сложения на эллиптической кривой

In [6]:
INF = Point(None, None, None) # точка на бесконечности

def extended_euclidean_algorithm(a, b):
    """
    Расширенный алгоритм евклида (см. 3 лаба по МОЗИ)
    """
    old_r, r = a, b
    old_s, s = 1, 0
    old_t, t = 0, 1
    while r != 0:
        quotient = old_r // r
        old_r, r = r, old_r - quotient * r
        old_s, s = s, old_s - quotient * s
        old_t, t = t, old_t - quotient * t
    return old_r, old_s, old_t

def inv(n, p):
    """ возвращает обратное число по модулю p: (n * m) % p == 1 """
    gcd, x, y = extended_euclidean_algorithm(n, p)
    return x % p

def elliptic_curve_addition(self, other: Point) -> Point:
    '''Операция сложения 2 точек на эллиптической кривой(см. https://habr.com/ru/post/335906/)'''
    if self == INF:
        return other
    if other == INF:
        return self

    if self.x == other.x and self.y != other.y:
        return INF
    
    if self.x == other.x:
        m = (3 * self.x**2 + self.curve.a) * inv(2 * self.y, self.curve.p)
    else:
        m = (self.y - other.y) * inv(self.x - other.x, self.curve.p)

    rx = (m**2 - self.x - other.x) % self.curve.p
    ry = (-(m*(rx - self.x) + self.y)) % self.curve.p
    return Point(self.curve, rx, ry)

Point.__add__ = elliptic_curve_addition # добавляем как операцию сложения, т.е. чтобы можно было просто писать +

Проверим сложение на простых примерах

In [7]:
# Проверим генерацию, sk - secret key, генерируем по нему public key
sk = 1
pk = G
print(f" secret key: {sk}\n public key: {(pk.x, pk.y)}")
print("Public key находится на нашей кривой?: ", (pk.y**2 - pk.x**3 - 7) % bitcoin_curve.p == 0)
# if it was 2, the public key is G + G:
sk = 2
pk = G + G
print(f" secret key: {sk}\n public key: {(pk.x, pk.y)}")
print("Public key находится на нашей кривой?: ", (pk.y**2 - pk.x**3 - 7) % bitcoin_curve.p == 0)
# etc.:
sk = 3
pk = G + G + G
print(f" secret key: {sk}\n public key: {(pk.x, pk.y)}")
print("Public key находится на нашей кривой?: ", (pk.y**2 - pk.x**3 - 7) % bitcoin_curve.p == 0)

 secret key: 1
 public key: (55066263022277343669578718895168534326250603453777594175500187360389116729240, 32670510020758816978083085130507043184471273380659243275938904335757337482424)
Public key находится на нашей кривой?:  True
 secret key: 2
 public key: (89565891926547004231252920425935692360644145829622209833684329913297188986597, 12158399299693830322967808612713398636155367887041628176798871954788371653930)
Public key находится на нашей кривой?:  True
 secret key: 3
 public key: (112711660439710606056748659173929673102114977341539408544630613555209775888121, 25583027980570883691656905877401976406448868254816295069919888960541586679410)
Public key находится на нашей кривой?:  True


Для повышения производительности необходимо оптимизировать алгоритм. Применим алгоритм удвоения-сложения. (см. https://habr.com/ru/post/335906/)

In [8]:
def double_and_add(self, k: int) -> Point:
    assert isinstance(k, int) and k >= 0
    result = INF
    append = self
    while k:
        if k & 1:
            result += append
        append += append
        k >>= 1
    return result

# добавляем как операцию умножения, т.е. чтобы можно было просто писать *
Point.__rmul__ = double_and_add

# проверим работоспособность
print(G == 1*G)
print(G + G == 2*G)
print(G + G + G == 3*G)

True
True
True


In [9]:
secret_key
public_key = secret_key * G
print("secret key: ",secret_key)
print(f"x: {public_key.x}\ny: {public_key.y}")
print("Public key находится на нашей кривой?: ", (public_key.y**2 - public_key.x**3 - 7) % bitcoin_curve.p == 0)

secret key:  88179880527772193127991559081690516360423762505570950046565861475646252139708
x: 2622435002776900012262913488300923425767213619137753162304988221125546485749
y: 42304050271267786694567464365427621221032123541303484276857170357560364141179
Public key находится на нашей кривой?:  True


Для шифрования sha256 будем использовать внешнюю библиотеку

In [10]:
from hashlib import sha256
print(sha256(b'here is a random bytes message, cool right?').hexdigest())
len(sha256(b'').digest())

69b9779edaa573a509999cbae415d3408c30544bad09727a1d64eff353c95b89


32

Создадим потом нам нужную функцию ripemd160

In [11]:
def gen_ripemd160_with_variable_scope_protector_to_not_pollute_global_namespace():

    import sys
    import struct

    # -----------------------------------------------------------------------------
    # public interface

    def ripemd160(b: bytes) -> bytes:
        """ simple wrapper for a simpler API to this hash function, just bytes to bytes """
        ctx = RMDContext()
        RMD160Update(ctx, b, len(b))
        digest = RMD160Final(ctx)
        return digest

    # -----------------------------------------------------------------------------

    class RMDContext:
        def __init__(self):
            self.state = [0x67452301, 0xEFCDAB89, 0x98BADCFE, 0x10325476, 0xC3D2E1F0] # uint32
            self.count = 0 # uint64
            self.buffer = [0]*64 # uchar

    def RMD160Update(ctx, inp, inplen):
        have = int((ctx.count // 8) % 64)
        inplen = int(inplen)
        need = 64 - have
        ctx.count += 8 * inplen
        off = 0
        if inplen >= need:
            if have:
                for i in range(need):
                    ctx.buffer[have+i] = inp[i]
                RMD160Transform(ctx.state, ctx.buffer)
                off = need
                have = 0
            while off + 64 <= inplen:
                RMD160Transform(ctx.state, inp[off:])
                off += 64
        if off < inplen:
            for i in range(inplen - off):
                ctx.buffer[have+i] = inp[off+i]

    def RMD160Final(ctx):
        size = struct.pack("<Q", ctx.count)
        padlen = 64 - ((ctx.count // 8) % 64)
        if padlen < 1 + 8:
            padlen += 64
        RMD160Update(ctx, PADDING, padlen-8)
        RMD160Update(ctx, size, 8)
        return struct.pack("<5L", *ctx.state)

    # -----------------------------------------------------------------------------

    K0 = 0x00000000
    K1 = 0x5A827999
    K2 = 0x6ED9EBA1
    K3 = 0x8F1BBCDC
    K4 = 0xA953FD4E
    KK0 = 0x50A28BE6
    KK1 = 0x5C4DD124
    KK2 = 0x6D703EF3
    KK3 = 0x7A6D76E9
    KK4 = 0x00000000

    PADDING = [0x80] + [0]*63

    def ROL(n, x):
        return ((x << n) & 0xffffffff) | (x >> (32 - n))

    def F0(x, y, z):
        return x ^ y ^ z

    def F1(x, y, z):
        return (x & y) | (((~x) % 0x100000000) & z)

    def F2(x, y, z):
        return (x | ((~y) % 0x100000000)) ^ z

    def F3(x, y, z):
        return (x & z) | (((~z) % 0x100000000) & y)

    def F4(x, y, z):
        return x ^ (y | ((~z) % 0x100000000))

    def R(a, b, c, d, e, Fj, Kj, sj, rj, X):
        a = ROL(sj, (a + Fj(b, c, d) + X[rj] + Kj) % 0x100000000) + e
        c = ROL(10, c)
        return a % 0x100000000, c

    def RMD160Transform(state, block): #uint32 state[5], uchar block[64]

        x = [0]*16
        assert sys.byteorder == 'little', "Only little endian is supported atm for RIPEMD160"
        x = struct.unpack('<16L', bytes(block[0:64]))

        a = state[0]
        b = state[1]
        c = state[2]
        d = state[3]
        e = state[4]

        #/* Round 1 */
        a, c = R(a, b, c, d, e, F0, K0, 11,  0, x)
        e, b = R(e, a, b, c, d, F0, K0, 14,  1, x)
        d, a = R(d, e, a, b, c, F0, K0, 15,  2, x)
        c, e = R(c, d, e, a, b, F0, K0, 12,  3, x)
        b, d = R(b, c, d, e, a, F0, K0,  5,  4, x)
        a, c = R(a, b, c, d, e, F0, K0,  8,  5, x)
        e, b = R(e, a, b, c, d, F0, K0,  7,  6, x)
        d, a = R(d, e, a, b, c, F0, K0,  9,  7, x)
        c, e = R(c, d, e, a, b, F0, K0, 11,  8, x)
        b, d = R(b, c, d, e, a, F0, K0, 13,  9, x)
        a, c = R(a, b, c, d, e, F0, K0, 14, 10, x)
        e, b = R(e, a, b, c, d, F0, K0, 15, 11, x)
        d, a = R(d, e, a, b, c, F0, K0,  6, 12, x)
        c, e = R(c, d, e, a, b, F0, K0,  7, 13, x)
        b, d = R(b, c, d, e, a, F0, K0,  9, 14, x)
        a, c = R(a, b, c, d, e, F0, K0,  8, 15, x) #/* #15 */
        #/* Round 2 */
        e, b = R(e, a, b, c, d, F1, K1,  7,  7, x)
        d, a = R(d, e, a, b, c, F1, K1,  6,  4, x)
        c, e = R(c, d, e, a, b, F1, K1,  8, 13, x)
        b, d = R(b, c, d, e, a, F1, K1, 13,  1, x)
        a, c = R(a, b, c, d, e, F1, K1, 11, 10, x)
        e, b = R(e, a, b, c, d, F1, K1,  9,  6, x)
        d, a = R(d, e, a, b, c, F1, K1,  7, 15, x)
        c, e = R(c, d, e, a, b, F1, K1, 15,  3, x)
        b, d = R(b, c, d, e, a, F1, K1,  7, 12, x)
        a, c = R(a, b, c, d, e, F1, K1, 12,  0, x)
        e, b = R(e, a, b, c, d, F1, K1, 15,  9, x)
        d, a = R(d, e, a, b, c, F1, K1,  9,  5, x)
        c, e = R(c, d, e, a, b, F1, K1, 11,  2, x)
        b, d = R(b, c, d, e, a, F1, K1,  7, 14, x)
        a, c = R(a, b, c, d, e, F1, K1, 13, 11, x)
        e, b = R(e, a, b, c, d, F1, K1, 12,  8, x) #/* #31 */
        #/* Round 3 */
        d, a = R(d, e, a, b, c, F2, K2, 11,  3, x)
        c, e = R(c, d, e, a, b, F2, K2, 13, 10, x)
        b, d = R(b, c, d, e, a, F2, K2,  6, 14, x)
        a, c = R(a, b, c, d, e, F2, K2,  7,  4, x)
        e, b = R(e, a, b, c, d, F2, K2, 14,  9, x)
        d, a = R(d, e, a, b, c, F2, K2,  9, 15, x)
        c, e = R(c, d, e, a, b, F2, K2, 13,  8, x)
        b, d = R(b, c, d, e, a, F2, K2, 15,  1, x)
        a, c = R(a, b, c, d, e, F2, K2, 14,  2, x)
        e, b = R(e, a, b, c, d, F2, K2,  8,  7, x)
        d, a = R(d, e, a, b, c, F2, K2, 13,  0, x)
        c, e = R(c, d, e, a, b, F2, K2,  6,  6, x)
        b, d = R(b, c, d, e, a, F2, K2,  5, 13, x)
        a, c = R(a, b, c, d, e, F2, K2, 12, 11, x)
        e, b = R(e, a, b, c, d, F2, K2,  7,  5, x)
        d, a = R(d, e, a, b, c, F2, K2,  5, 12, x) #/* #47 */
        #/* Round 4 */
        c, e = R(c, d, e, a, b, F3, K3, 11,  1, x)
        b, d = R(b, c, d, e, a, F3, K3, 12,  9, x)
        a, c = R(a, b, c, d, e, F3, K3, 14, 11, x)
        e, b = R(e, a, b, c, d, F3, K3, 15, 10, x)
        d, a = R(d, e, a, b, c, F3, K3, 14,  0, x)
        c, e = R(c, d, e, a, b, F3, K3, 15,  8, x)
        b, d = R(b, c, d, e, a, F3, K3,  9, 12, x)
        a, c = R(a, b, c, d, e, F3, K3,  8,  4, x)
        e, b = R(e, a, b, c, d, F3, K3,  9, 13, x)
        d, a = R(d, e, a, b, c, F3, K3, 14,  3, x)
        c, e = R(c, d, e, a, b, F3, K3,  5,  7, x)
        b, d = R(b, c, d, e, a, F3, K3,  6, 15, x)
        a, c = R(a, b, c, d, e, F3, K3,  8, 14, x)
        e, b = R(e, a, b, c, d, F3, K3,  6,  5, x)
        d, a = R(d, e, a, b, c, F3, K3,  5,  6, x)
        c, e = R(c, d, e, a, b, F3, K3, 12,  2, x) #/* #63 */
        #/* Round 5 */
        b, d = R(b, c, d, e, a, F4, K4,  9,  4, x)
        a, c = R(a, b, c, d, e, F4, K4, 15,  0, x)
        e, b = R(e, a, b, c, d, F4, K4,  5,  5, x)
        d, a = R(d, e, a, b, c, F4, K4, 11,  9, x)
        c, e = R(c, d, e, a, b, F4, K4,  6,  7, x)
        b, d = R(b, c, d, e, a, F4, K4,  8, 12, x)
        a, c = R(a, b, c, d, e, F4, K4, 13,  2, x)
        e, b = R(e, a, b, c, d, F4, K4, 12, 10, x)
        d, a = R(d, e, a, b, c, F4, K4,  5, 14, x)
        c, e = R(c, d, e, a, b, F4, K4, 12,  1, x)
        b, d = R(b, c, d, e, a, F4, K4, 13,  3, x)
        a, c = R(a, b, c, d, e, F4, K4, 14,  8, x)
        e, b = R(e, a, b, c, d, F4, K4, 11, 11, x)
        d, a = R(d, e, a, b, c, F4, K4,  8,  6, x)
        c, e = R(c, d, e, a, b, F4, K4,  5, 15, x)
        b, d = R(b, c, d, e, a, F4, K4,  6, 13, x) #/* #79 */

        aa = a
        bb = b
        cc = c
        dd = d
        ee = e

        a = state[0]
        b = state[1]
        c = state[2]
        d = state[3]
        e = state[4]

        #/* Parallel round 1 */
        a, c = R(a, b, c, d, e, F4, KK0,  8,  5, x)
        e, b = R(e, a, b, c, d, F4, KK0,  9, 14, x)
        d, a = R(d, e, a, b, c, F4, KK0,  9,  7, x)
        c, e = R(c, d, e, a, b, F4, KK0, 11,  0, x)
        b, d = R(b, c, d, e, a, F4, KK0, 13,  9, x)
        a, c = R(a, b, c, d, e, F4, KK0, 15,  2, x)
        e, b = R(e, a, b, c, d, F4, KK0, 15, 11, x)
        d, a = R(d, e, a, b, c, F4, KK0,  5,  4, x)
        c, e = R(c, d, e, a, b, F4, KK0,  7, 13, x)
        b, d = R(b, c, d, e, a, F4, KK0,  7,  6, x)
        a, c = R(a, b, c, d, e, F4, KK0,  8, 15, x)
        e, b = R(e, a, b, c, d, F4, KK0, 11,  8, x)
        d, a = R(d, e, a, b, c, F4, KK0, 14,  1, x)
        c, e = R(c, d, e, a, b, F4, KK0, 14, 10, x)
        b, d = R(b, c, d, e, a, F4, KK0, 12,  3, x)
        a, c = R(a, b, c, d, e, F4, KK0,  6, 12, x) #/* #15 */
        #/* Parallel round 2 */
        e, b = R(e, a, b, c, d, F3, KK1,  9,  6, x)
        d, a = R(d, e, a, b, c, F3, KK1, 13, 11, x)
        c, e = R(c, d, e, a, b, F3, KK1, 15,  3, x)
        b, d = R(b, c, d, e, a, F3, KK1,  7,  7, x)
        a, c = R(a, b, c, d, e, F3, KK1, 12,  0, x)
        e, b = R(e, a, b, c, d, F3, KK1,  8, 13, x)
        d, a = R(d, e, a, b, c, F3, KK1,  9,  5, x)
        c, e = R(c, d, e, a, b, F3, KK1, 11, 10, x)
        b, d = R(b, c, d, e, a, F3, KK1,  7, 14, x)
        a, c = R(a, b, c, d, e, F3, KK1,  7, 15, x)
        e, b = R(e, a, b, c, d, F3, KK1, 12,  8, x)
        d, a = R(d, e, a, b, c, F3, KK1,  7, 12, x)
        c, e = R(c, d, e, a, b, F3, KK1,  6,  4, x)
        b, d = R(b, c, d, e, a, F3, KK1, 15,  9, x)
        a, c = R(a, b, c, d, e, F3, KK1, 13,  1, x)
        e, b = R(e, a, b, c, d, F3, KK1, 11,  2, x) #/* #31 */
        #/* Parallel round 3 */
        d, a = R(d, e, a, b, c, F2, KK2,  9, 15, x)
        c, e = R(c, d, e, a, b, F2, KK2,  7,  5, x)
        b, d = R(b, c, d, e, a, F2, KK2, 15,  1, x)
        a, c = R(a, b, c, d, e, F2, KK2, 11,  3, x)
        e, b = R(e, a, b, c, d, F2, KK2,  8,  7, x)
        d, a = R(d, e, a, b, c, F2, KK2,  6, 14, x)
        c, e = R(c, d, e, a, b, F2, KK2,  6,  6, x)
        b, d = R(b, c, d, e, a, F2, KK2, 14,  9, x)
        a, c = R(a, b, c, d, e, F2, KK2, 12, 11, x)
        e, b = R(e, a, b, c, d, F2, KK2, 13,  8, x)
        d, a = R(d, e, a, b, c, F2, KK2,  5, 12, x)
        c, e = R(c, d, e, a, b, F2, KK2, 14,  2, x)
        b, d = R(b, c, d, e, a, F2, KK2, 13, 10, x)
        a, c = R(a, b, c, d, e, F2, KK2, 13,  0, x)
        e, b = R(e, a, b, c, d, F2, KK2,  7,  4, x)
        d, a = R(d, e, a, b, c, F2, KK2,  5, 13, x) #/* #47 */
        #/* Parallel round 4 */
        c, e = R(c, d, e, a, b, F1, KK3, 15,  8, x)
        b, d = R(b, c, d, e, a, F1, KK3,  5,  6, x)
        a, c = R(a, b, c, d, e, F1, KK3,  8,  4, x)
        e, b = R(e, a, b, c, d, F1, KK3, 11,  1, x)
        d, a = R(d, e, a, b, c, F1, KK3, 14,  3, x)
        c, e = R(c, d, e, a, b, F1, KK3, 14, 11, x)
        b, d = R(b, c, d, e, a, F1, KK3,  6, 15, x)
        a, c = R(a, b, c, d, e, F1, KK3, 14,  0, x)
        e, b = R(e, a, b, c, d, F1, KK3,  6,  5, x)
        d, a = R(d, e, a, b, c, F1, KK3,  9, 12, x)
        c, e = R(c, d, e, a, b, F1, KK3, 12,  2, x)
        b, d = R(b, c, d, e, a, F1, KK3,  9, 13, x)
        a, c = R(a, b, c, d, e, F1, KK3, 12,  9, x)
        e, b = R(e, a, b, c, d, F1, KK3,  5,  7, x)
        d, a = R(d, e, a, b, c, F1, KK3, 15, 10, x)
        c, e = R(c, d, e, a, b, F1, KK3,  8, 14, x) #/* #63 */
        #/* Parallel round 5 */
        b, d = R(b, c, d, e, a, F0, KK4,  8, 12, x)
        a, c = R(a, b, c, d, e, F0, KK4,  5, 15, x)
        e, b = R(e, a, b, c, d, F0, KK4, 12, 10, x)
        d, a = R(d, e, a, b, c, F0, KK4,  9,  4, x)
        c, e = R(c, d, e, a, b, F0, KK4, 12,  1, x)
        b, d = R(b, c, d, e, a, F0, KK4,  5,  5, x)
        a, c = R(a, b, c, d, e, F0, KK4, 14,  8, x)
        e, b = R(e, a, b, c, d, F0, KK4,  6,  7, x)
        d, a = R(d, e, a, b, c, F0, KK4,  8,  6, x)
        c, e = R(c, d, e, a, b, F0, KK4, 13,  2, x)
        b, d = R(b, c, d, e, a, F0, KK4,  6, 13, x)
        a, c = R(a, b, c, d, e, F0, KK4,  5, 14, x)
        e, b = R(e, a, b, c, d, F0, KK4, 15,  0, x)
        d, a = R(d, e, a, b, c, F0, KK4, 13,  3, x)
        c, e = R(c, d, e, a, b, F0, KK4, 11,  9, x)
        b, d = R(b, c, d, e, a, F0, KK4, 11, 11, x) #/* #79 */

        t = (state[1] + cc + d) % 0x100000000
        state[1] = (state[2] + dd + e) % 0x100000000
        state[2] = (state[3] + ee + a) % 0x100000000
        state[3] = (state[4] + aa + b) % 0x100000000
        state[4] = (state[0] + bb + c) % 0x100000000
        state[0] = t % 0x100000000
    
    return ripemd160
    
ripemd160 = gen_ripemd160_with_variable_scope_protector_to_not_pollute_global_namespace()
print(ripemd160(b'hello this is a test').hex())
print("number of bytes in a RIPEMD-160 digest: ", len(ripemd160(b'')))

f51960af7dd4813a587ab26388ddab3b28d1f7b4
number of bytes in a RIPEMD-160 digest:  20


В данном классе мы получаем наш биткоин адресс. 

In [12]:
class PublicKey(Point):
    """
    Публичный ключ это просто точка на кривой, к которой добавлена
    дополнительная кодирующая/декодирующая функцио(а)нальность.
    """

    @classmethod
    def from_point(cls, pt: Point):
        """ превращаем точку в public key """
        return cls(pt.curve, pt.x, pt.y)

    def encode(self, compressed, hash160=False):
        """ Кодировка публичного ключа """
        if compressed:
            # (x,y) слишком много занимает места. 
            # вместо этого кодируем только x, а чтобы различить какой y
            # добавляем байт кодировки если y(x) четное, то 0x02; если y(x) нечетное, то 0x03
            prefix = b'\x02' if self.y % 2 == 0 else b'\x03'
            pkb = prefix + self.x.to_bytes(32, 'big')
        else:
            pkb = b'\x04' + self.x.to_bytes(32, 'big') + self.y.to_bytes(32, 'big')
        # если захотим будет наш хэш
        return ripemd160(sha256((pkb)).digest()) if hash160 else pkb

    def address(self, net: str, compressed: bool) -> str:
        """ возвращаем биткоин адресс который ассоциируется с этом public key """
        # кодируем функцией выше
        pkb_hash = self.encode(compressed=compressed, hash160=True)
        # добавляем байт версии: Main - 0x00; Test - 0x6f
        version = {'main': b'\x00', 'test': b'\x6f'}
        ver_pkb_hash = version[net] + pkb_hash
        # считаем контрольную сумму
        checksum = sha256(sha256(ver_pkb_hash).digest()).digest()[:4]
        # добавляем к адресу чтобы сформировать 25-битный биткоин адресс
        byte_address = ver_pkb_hash + checksum
        # кодируем результат с помощью b58 encode
        b58check_address = b58encode(byte_address)
        return b58check_address


b58encode - кодировка Base58. Ее суть в том, чтобы максимально кратко записать последовательность байт в удобочитаемом формате и при этом сделать вероятность возможных опечаток еще меньше.

In [13]:
# алгоритм был взять от сюда: https://en.bitcoin.it/wiki/Base58Check_encoding

alphabet = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'

def b58encode(b: bytes) -> str:
    assert len(b) == 25 
    n = int.from_bytes(b, 'big')
    chars = []
    while n:
        n, i = divmod(n, 58)
        chars.append(alphabet[i])
    num_leading_zeros = len(b) - len(b.lstrip(b'\x00'))
    res = num_leading_zeros * alphabet[0] + ''.join(reversed(chars))
    return res


Сейчас посмотрим на наш биткоин адресс:

In [14]:
#напоминание о том, что такое public key
#secret_key
#public_key = secret_key * G
print("secret key: ",secret_key)
print(f"x: {public_key.x}\ny: {public_key.y}")
address = PublicKey.from_point(public_key).address(net='test', compressed=True)
print("Биткоин адресс: ",address)

secret key:  88179880527772193127991559081690516360423762505570950046565861475646252139708
x: 2622435002776900012262913488300923425767213619137753162304988221125546485749
y: 42304050271267786694567464365427621221032123541303484276857170357560364141179
Биткоин адресс:  mrhQf5d7CuasW4SY1qeeQK5wWpCcrajrCj


Проверить использовался ли этот кошель можно здесь. Так как это тест, то все деньги не настоящие:
[https://www.blockchain.com/btc-testnet/address/moANLaHtXoEbBh6j6JFUoVrzbYwhAFuGdE](https://www.blockchain.com/btc-testnet/address/moANLaHtXoEbBh6j6JFUoVrzbYwhAFuGdE).
И конечно мы каждый раз будем получать разные результаты, поэтому воспользуемся самым простым private key.

In [15]:
lol_secret_key = 1
lol_public_key = lol_secret_key * G
lol_address = PublicKey.from_point(lol_public_key).address(net='test', compressed=True)
lol_address

'mrCDrCybB6J1vRfbwM5hemdJz73FwDBC8r'

In [23]:
secret_key = 73550845895012693914846049734175534669350379867969770173435494349140675121036 #здесь есть деньги и их не надо
# будет запрашивать
public_key = secret_key * G
address = PublicKey.from_point(public_key).address(net='test', compressed=True)
print(hex(secret_key))
print(PublicKey.from_point(public_key).encode(compressed=True, hash160=True).hex())
print("Our first Bitcoin identity:")
print("1. secret key: ", secret_key)
print("2. public key: ", (public_key.x, public_key.y))
print("3. Bitcoin address: ", address)

0xa29c4db3d20833556cecc2581bfa530d5d685155e98cd7d9dd23f7b6a801cf8c
53dc576f69a8cc263f688dd80b27d6dd84421c47
Our first Bitcoin identity:
1. secret key:  73550845895012693914846049734175534669350379867969770173435494349140675121036
2. public key:  (75381168214966211376566701773070236611376060524760107291010051445604727593886, 108836286570701899840289658238037216752223007433477180898362291967973334190961)
3. Bitcoin address:  moANLaHtXoEbBh6j6JFUoVrzbYwhAFuGdE


In [17]:
secret_key2 = random.randrange(1, bitcoin_gen.n)
assert 1 <= secret_key2 < bitcoin_gen.n # проверяем валидность
secret_key2 = 2064071921415314472189084935496006784787091186969848311062728790402231534394 #0 баланс, сюда перекинем
public_key2 = secret_key2 * G
address2 = PublicKey.from_point(public_key2).address(net='test', compressed=True)

print(PublicKey.from_point(public_key2).encode(compressed=True, hash160=True).hex())
print("Our second Bitcoin identity:")
print("1. secret key: ", secret_key2)
print("2. public key: ", (public_key2.x, public_key2.y))
print("3. Bitcoin address: ", address2)

d37057c9abbd6ccb692b41c1633210cd3662c933
Our second Bitcoin identity:
1. secret key:  2064071921415314472189084935496006784787091186969848311062728790402231534394
2. public key:  (14728729335422913652898378386999460579207077113556725147938307604202228623964, 70731673037988858481120694139580083258046390705045894940198884664555711235280)
3. Bitcoin address:  mznwTFe8jAD5XFTb1wihXRu2DJN36y2uj2


### Попытка сделать собственный код транзакции

In [263]:
!pip install ecdsa

  Using cached ecdsa-0.17.0-py2.py3-none-any.whl (119 kB)


In [18]:
import struct, hashlib, ecdsa
import binascii


def makeaman(strin):
    return (' '.join(['/x%02x' % b for b in b'strin'])).replace("/",chr(92)).replace(" ","").encode()


class raw_tx:
    version      = struct.pack("<L", 1)
    tx_in_count  = struct.pack("<B", 1)
    tx_in        = {} #TEMP
    tx_out_count = struct.pack("<B", 2)
    tx_out1      = {} #TEMP
    tx_out2      = {} #TEMP
    lock_time    = struct.pack("<L", 0)

def flip_byte_order(string):
    flipped = "".join(reversed([string[i:i+2] for i in range(0, len(string), 2)]))
    return flipped

rtx = raw_tx()

rtx.tx_in["txouthash"] = binascii.unhexlify(flip_byte_order("2c26f33507672aa4c24f97a02881794e430fc3554c3ffad75b7dc8b753ec7277"))
rtx.tx_in["tx_out_index"] = struct.pack("<L", 0)
rtx.tx_in["script"] = binascii.unhexlify(("76a914%s88ac" % "53dc576f69a8cc263f688dd80b27d6dd84421c47"))
rtx.tx_in["scrip_bytes"] = struct.pack("<B", len(rtx.tx_in["script"]))
rtx.tx_in["sequence"] = binascii.unhexlify("ffffffff")

rtx.tx_out1["value"] = struct.pack("<Q", 50000)
rtx.tx_out1["pk_script"]  = binascii.unhexlify("76a914%s88ac" % "d37057c9abbd6ccb692b41c1633210cd3662c933")
rtx.tx_out1["pk_script_bytes"] = struct.pack("<B", len(rtx.tx_out1["pk_script"]))

rtx.tx_out2["value"] = struct.pack("<Q", 47500)
rtx.tx_out2["pk_script"]  = binascii.unhexlify("76a914%s88ac" % "53dc576f69a8cc263f688dd80b27d6dd84421c47")
rtx.tx_out2["pk_script_bytes"] = struct.pack("<B", len(rtx.tx_out2["pk_script"]))

raw_tx_string = (

    rtx.version
    + rtx.tx_in_count
    + rtx.tx_in["txouthash"]
    + rtx.tx_in["tx_out_index"]
    + rtx.tx_in["scrip_bytes"]
    + rtx.tx_in["script"]
    + rtx.tx_in["sequence"]
    + rtx.tx_out_count
    + rtx.tx_out1["value"]
    + rtx.tx_out1["pk_script_bytes"]
    + rtx.tx_out1["pk_script"]
    + rtx.tx_out2["value"]
    + rtx.tx_out2["pk_script_bytes"]
    + rtx.tx_out2["pk_script"]
    + rtx.lock_time
    + struct.pack("<L", 1)
)

hashed_tx_to_sign = sha256(sha256(raw_tx_string).digest()).digest()

sk = ecdsa.SigningKey.from_string(binascii.unhexlify('a29c4db3d20833556cecc2581bfa530d5d685155e98cd7d9dd23f7b6a801cf8c'), curve = ecdsa.SECP256k1)

vk = sk.verifying_key

#print(binascii.unhexlify('a29c4db3d20833556cecc2581bfa530d5d685155e98cd7d9dd23f7b6a801cf8c'))
#bytes.fromhex('a29c4db3d20833556cecc2581bfa530d5d685155e98cd7d9dd23f7b6a801cf8c')

public_key = (binascii.unhexlify('04')+ vk.to_string())

signature = sk.sign_digest(hashed_tx_to_sign, sigencode = ecdsa.util.sigencode_der_canonize)

sigscript = (
    signature
    + binascii.unhexlify("01")
    + struct.pack("<B", len(public_key))
    + public_key
)

real_tx = (
    rtx.version
    + rtx.tx_in_count
    + rtx.tx_in["txouthash"]
    + rtx.tx_in["tx_out_index"]
    + struct.pack("<B", len(sigscript) + 1)
    + struct.pack("<B", len(signature) + 1)
    + sigscript
    + rtx.tx_in["sequence"]
    + rtx.tx_out_count
    + rtx.tx_out1["value"]
    + rtx.tx_out1["pk_script_bytes"]
    + rtx.tx_out1["pk_script"]
    + rtx.tx_out2["value"]
    + rtx.tx_out2["pk_script_bytes"]
    + rtx.tx_out2["pk_script"]
    + rtx.lock_time
)

print(real_tx)

b"\x01\x00\x00\x00\x01wr\xecS\xb7\xc8}[\xd7\xfa?LU\xc3\x0fCNy\x81(\xa0\x97O\xc2\xa4*g\x075\xf3&,\x00\x00\x00\x00\x8aG0D\x02 Mk\x80\xa0&\xe1\x00'\x92\x9e\xb3\xa6\xe4\xfaMJ\x0eR\xe5\x8f\xbe\xee\x82\x8a\x0fS\xbb\x8f\xe0\xbf\xfa\x19\x02 \x7f=P\xb3aq\xee!V\xe9\xf5\xda8\xaf\xe3\x10\x15\xa1^Z\xd6\x934\x8e\xe8P\xfa\xbe\xaa'\xb1\x08\x01A\x04\xa6\xa8:\xaf\xe3\xba\x16!\xbf\xd81A\xc2\x88\xae\x95,b\xaf>\x928\xea;\xda%!g\xe7\xd3\x1b\x9e\xf0\x9f'\xbb\xa9\x06\xfc\xe4\xc1g\x98B!f\xd9\x83\xf0\xde\x95\xbc\xdfmb5\xda\x9fGr\x85\x08\x0bq\xff\xff\xff\xff\x02P\xc3\x00\x00\x00\x00\x00\x00\x19v\xa9\x14\xd3pW\xc9\xab\xbdl\xcbi+A\xc1c2\x10\xcd6b\xc93\x88\xac\x8c\xb9\x00\x00\x00\x00\x00\x00\x19v\xa9\x14S\xdcWoi\xa8\xcc&?h\x8d\xd8\x0b'\xd6\xdd\x84B\x1cG\x88\xac\x00\x00\x00\x00"


In [19]:
real_tx.hex()

'01000000017772ec53b7c87d5bd7fa3f4c55c30f434e798128a0974fc2a42a670735f3262c000000008a47304402204d6b80a026e10027929eb3a6e4fa4d4a0e52e58fbeee828a0f53bb8fe0bffa1902207f3d50b36171ee2156e9f5da38afe31015a15e5ad693348ee850fabeaa27b108014104a6a83aafe3ba1621bfd83141c288ae952c62af3e9238ea3bda252167e7d31b9ef09f27bba906fce4c16798422166d983f0de95bcdf6d6235da9f477285080b71ffffffff0250c30000000000001976a914d37057c9abbd6ccb692b41c1633210cd3662c93388ac8cb90000000000001976a91453dc576f69a8cc263f688dd80b27d6dd84421c4788ac00000000'

## Транзанкция с помощью внешней библиотеки

In [119]:
pip install bit

Note: you may need to restart the kernel to use updated packages.


#### Если что выписать себе немного биткоинов для теста можно здесь https://testnet-faucet.mempool.co/

In [120]:
from bit import PrivateKeyTestnet

my_key = PrivateKeyTestnet("cV6iiXiQLW9oqjGt85r9CVj1vgdB5eJ8jye7P7DT7sGyCYcDxxJq")

print(my_key.version)

print(my_key.to_wif())

print(my_key.address)
#cV6iiXiQLW9oqjGt85r9CVj1vgdB5eJ8jye7P7DT7sGyCYcDxxJq
#muGYcQ3ntZqdoDPKxTtsPzfV586sNA1oYM

test
cV6iiXiQLW9oqjGt85r9CVj1vgdB5eJ8jye7P7DT7sGyCYcDxxJq
muGYcQ3ntZqdoDPKxTtsPzfV586sNA1oYM


In [121]:
print(my_key.balance_as('usd'))

0


In [122]:
tx_hash = my_key.send([('mgh4VjZx5MpkHRis9mDsF2ZcKLdXoP3oQ4', 1, 'usd')])

print(tx_hash)

599dc4bddecff1d5463f012063df0e1104eaa9350bd872a0bbd4eec283239c8b
